In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
data = pd.read_csv("./data/sft/NSL-KDD-100000-sft.csv")
# data = pd.read_csv("./data/sft/Mixed-sft-500000.csv")
# 划分训练集和验证集
train_texts, eval_texts, train_labels, eval_labels = train_test_split(
    data["flow"].tolist(), data["class"].tolist(), test_size=0.2, random_state=42
)
# 转换为 Hugging Face Dataset 格式
dataset = DatasetDict({
    "train": Dataset.from_dict({"text": train_texts, "label": train_labels}),
    "eval": Dataset.from_dict({"text": eval_texts, "label": eval_labels})
})

In [2]:
# 遍历数据，找到最长文本的长度（基于逗号分词）
text_lengths = [len(text.split(",")) for text in data["flow"].tolist()]
max_length = max(text_lengths)
print(f"实际设定的 max_length: {max_length}")

实际设定的 max_length: 41


In [3]:
from transformers import BertTokenizer
# 加载 BERT 分词器
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# 逗号分词 + 重新转换为 BERT 需要的 `input_ids`
def custom_tokenize_function(examples):
    # 逗号分词
    tokenized_texts = [text.split(",") for text in examples["text"]]
    # 将每个短语转换为 BERT input_ids
    encodings = tokenizer(
        tokenized_texts,  # 逗号分词后的文本
        padding="max_length",
        max_length=max_length,
        truncation=True,
        is_split_into_words=True  # 关键参数：告诉 tokenizer 文本已经被手动分词
    )
    return encodings

In [4]:
tokenized_datasets = dataset.map(custom_tokenize_function, batched=True)
train_dataset = tokenized_datasets["train"].shuffle(seed=42)
eval_dataset = tokenized_datasets["eval"].shuffle(seed=42)

Map:   0%|          | 0/80000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [5]:
from transformers import AutoModelForSequenceClassification
base_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2, hidden_dropout_prob=0.2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from peft import LoraConfig, get_peft_model
# **添加 LoRA 适配**
lora_config = LoraConfig(
    r=8,  # LoRA 低秩矩阵的秩
    lora_alpha=16,  # LoRA scaling factor
    lora_dropout=0.1,  # Dropout 防止过拟合
    target_modules=["query", "value"],  # 仅优化 Query 和 Value 层
    task_type="SEQ_CLS",  # 序列分类任务
)
# 将 LoRA 集成到 BERT 模型
lora_model = get_peft_model(base_model, lora_config)
lora_model.print_trainable_parameters()  # 查看可训练参数

trainable params: 296,450 || all params: 109,780,228 || trainable%: 0.2700


In [7]:
# 继续训练模型
lora_model.train()

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.2, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default

In [8]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
# 计算评估指标
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="binary")
    return {
        "eval_loss": float(np.mean(logits)),  # 确保 `eval_loss` 存在
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

In [9]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
training_args = TrainingArguments(
    # output_dir="test_trainer-sft-diy-lora-mixed",
    output_dir="./trainer/test_trainer-sft-diy-lora",
    eval_strategy="epoch",
    save_strategy="epoch",  # 每个 epoch 保存一次模型
    save_steps=None,  # 取消按steps保存
    learning_rate=2e-5,  # 学习率  2e-5    5e-5
    per_device_train_batch_size=64,  # 适当增加 batch_size，默认 8
    per_device_eval_batch_size=64,
    num_train_epochs=5,  # 降低训练轮数，避免过拟合
    weight_decay=0.02,  # 加入 L2 正则化
    load_best_model_at_end=True,  # 训练结束后加载最佳模型
    logging_strategy="epoch",  # 确保每个 epoch 打印 loss
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],  # 早停机制
)

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [10]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.229600,0.123907,0.967200,0.949030,0.991424,0.969764
2,0.106000,0.070093,0.979700,0.964497,0.998492,0.981200
3,0.071800,0.049900,0.983550,0.974963,0.994534,0.984651
4,0.061400,0.046476,0.986000,0.978509,0.995476,0.986920
5,0.055800,0.047341,0.984850,0.975111,0.996890,0.985880


TrainOutput(global_step=6250, training_loss=0.10491847045898438, metrics={'train_runtime': 792.6451, 'train_samples_per_second': 504.639, 'train_steps_per_second': 7.885, 'total_flos': 8456946672000000.0, 'train_loss': 0.10491847045898438, 'epoch': 5.0})